### Basic building block
Just printing out what a logic does...

First let’s define a simple class for storing our statements.


In [36]:
class Statement:
    def __init__(self,subject,predicate,singular=True):
        self.subject = subject
        self.predicate = predicate
        self.singular = singular

Now we will define our actual relation class with methods to pring natural language statements from logic...

In [37]:
class Relation:
    def __init__(self):
        self.map = {}

    def add(self, statement, subject, predicate, singular=True):
        self.map[statement] = Statement(subject, predicate, singular)

    def print_statement(self, statement):
        if self.map[statement].singular:
            print(self.map[statement].subject, " is ", self.map[statement].predicate, end="")
        else:
            print(self.map[statement].subject, " are ", self.map[statement].predicate, end="")

    def print_neg_statement(self, statement):
        if self.map[statement].singular:
            print(self.map[statement].subject, " is not ", self.map[statement].predicate, end="")
        else:
            print(self.map[statement].subject, " are not ", self.map[statement].predicate, end="")

    def build_statement(self, logic):
        """
        & = AND
        | = OR
        ! = NOT
        ~ = IMPLIES
        * = IF AND ONLY IF

        A = EVERY
        E = SOME
        """
        logic = logic.replace(" ", "")  # no space
        logic = logic.replace(",", "")  # no comma

        neg = False
        only_sub = False

        for i in range(len(logic)):
            if logic[i] == '!':
                neg = True
            elif logic[i] == '&':
                print(" and ", end="")
            elif logic[i] == '|':
                print(" or ", end="")
            elif logic[i] == '*':
                print(" if and only if, ", end="")
            elif logic[i] == '~':
                print(" implies, ", end="")
            elif logic[i] == 'A':
                print(" For all ", end="")
                only_sub = True
            elif logic[i] == 'E':
                print(" For some ", end="")
                only_sub = True
            elif logic[i] in ['(', ')', ',']:
                continue
            else:
                if only_sub:
                    print(self.map[logic[i]].subject, end=", ")
                    only_sub = False
                elif neg:
                    self.print_neg_statement(logic[i])
                    neg = False
                else:
                    self.print_statement(logic[i])
        print(".")

Let’s test our class,

In [38]:
relation = Relation()
relation.add("s","students","brilliant",singular=False)
relation.add("a","Tanmoy","bad boy")
relation.add("b","Tanmoy","lazy")
relation.add("c","Prosenjit","friend of Tanmoy")

relation.build_statement("A(s) s")
relation.build_statement("a & !b ~ a")

 For all students, students  are  brilliant.
Tanmoy  is  bad boy and Tanmoy  is not  lazy implies, Tanmoy  is  bad boy.


### Truth Table
Let’s build truth tables for basic logical operations. We need to draw tables again and again, so let’s create a function to do that.

In [42]:
from itertools import product

def truth_table(express):
    print(" P | Q | R | r |\n ______________")
    for P,Q,R in product([0,1], repeat=3):
        r=express(P,Q,R)
        print(f" {P} | {Q} | {R} | {r}")
express=lambda P,Q,R: P and Q and R
truth_table(express)

 P | Q | R | r |
 ______________
 0 | 0 | 0 | 0
 0 | 0 | 1 | 0
 0 | 1 | 0 | 0
 0 | 1 | 1 | 0
 1 | 0 | 0 | 0
 1 | 0 | 1 | 0
 1 | 1 | 0 | 0
 1 | 1 | 1 | 1


### Relation Builder (BONUS)
Suppose we have a list of relations like, someone is someones parent or children and we want to build relation chains like grandparent, grandchild, cousin etc. We can do that with a simple class.

In [43]:
class People:
    def __init__(self, name, parent=None, children=None):
        self.name = name
        self.parent = parent
        self.children = [children]

class Relation:
    def __init__(self):
        self.map = {}

    def add(self, person):
        self.map[person.name] = person

    def parent(self, parent, child):
        self.map[child].parent = parent
        if self.map[parent].children == [None]:
            self.map[parent].children = [child]
        else:
            self.map[parent].children.append(child)

    def find_grandparent(self, name):
        parent = self.map[name].parent
        if parent is None:
            return None
        grandparent = self.map[parent].parent
        return grandparent

In [44]:
relation = Relation()

john = People("John")
toy = People("Toy")
mary = People("Mary")
joe = People("Joe")

relation.add(john)
relation.add(toy)
relation.add(mary)
relation.add(joe)

relation.parent("John", "Toy")
relation.parent("Toy", "Mary")
relation.parent("Mary", "Joe")

print("Mary's grandparent is:", relation.find_grandparent("Mary"))

Mary's grandparent is: John
